In [1]:
!python --version

Python 3.10.9


In [2]:
!nvidia-smi

Tue Jan 31 17:22:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P8    14W /  70W |      2MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


In [4]:
import torch
torch.__version__

'2.0.0.dev20230125'

In [5]:
import transformers
transformers.__version__

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'4.22.2'

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch._dynamo as torchdynamo
import torch

In [7]:
# default cache size needs to be increased to store the many graphs with generative models
torchdynamo.config.cache_size_limit = 512

In [8]:
model_name = "../coeus_inference/hf_models/query_classify_v2"

In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.eval().cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [10]:
inputs = tokenizer("Generate taxonomy for query: dildo", return_tensors="pt", padding='max_length', truncation=True, max_length=50).to('cuda')

# dynamo

In [11]:
model.generate2 = torchdynamo.optimize("inductor")(model.generate)

In [12]:
# dynamo warm up
print(tokenizer.batch_decode(model.generate2(**inputs)))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/opt/conda/lib/python3.10/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']


In [13]:
%%timeit
print(tokenizer.batch_decode(model.generate2(**inputs)))

/opt/conda/lib/python3.10/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex 

In [14]:
inputs2 = tokenizer("Generate taxonomy for query: women gucci", return_tensors="pt", padding='max_length', truncation=True, max_length=50).to('cuda')

In [19]:
%%timeit
print(tokenizer.batch_decode(model.generate2(**inputs2)))

["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's bags</s>"]
["<pad> luggage & bags > women's

In [16]:
inputs3 = tokenizer("Generate taxonomy for query: baby milk", return_tensors="pt", padding='max_length', truncation=True, max_length=50).to('cuda')

In [29]:
%%timeit
print(tokenizer.batch_decode(model.generate2(**inputs3)))

/opt/conda/lib/python3.10/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > feeding > bottle feeding > baby food storage</s>']
['<pad> mother & kids > f

# constrained

In [20]:
from typing import List, Dict
class Trie(object):
    def __init__(self, sequences: List[List[int]] = []):
        self.trie_dict = {}
        self.len = 0
        if sequences:
            for sequence in sequences:
                Trie._add_to_trie(sequence, self.trie_dict)
                self.len += 1

        self.append_trie = None
        self.bos_token_id = None

    def append(self, trie, bos_token_id):
        self.append_trie = trie
        self.bos_token_id = bos_token_id

    def add(self, sequence: List[int]):
        Trie._add_to_trie(sequence, self.trie_dict)
        self.len += 1

    def get(self, prefix_sequence: List[int]):
        return Trie._get_from_trie(
            prefix_sequence, self.trie_dict, self.append_trie, self.bos_token_id
        )

    @staticmethod
    def load_from_dict(trie_dict):
        trie = Trie()
        trie.trie_dict = trie_dict
        trie.len = sum(1 for _ in trie)
        return trie

    @staticmethod
    def _add_to_trie(sequence: List[int], trie_dict: Dict):
        if sequence:
            if sequence[0] not in trie_dict:
                trie_dict[sequence[0]] = {}
            Trie._add_to_trie(sequence[1:], trie_dict[sequence[0]])

    @staticmethod
    def _get_from_trie(
        prefix_sequence: List[int],
        trie_dict: Dict,
        append_trie=None,
        bos_token_id: int = None,
    ):
        if len(prefix_sequence) == 0:
            output = list(trie_dict.keys())
            if append_trie and bos_token_id in output:
                output.remove(bos_token_id)
                output += list(append_trie.trie_dict.keys())
            return output
        elif prefix_sequence[0] in trie_dict:
            return Trie._get_from_trie(
                prefix_sequence[1:],
                trie_dict[prefix_sequence[0]],
                append_trie,
                bos_token_id,
            )
        else:
            if append_trie:
                return append_trie.get(prefix_sequence)
            else:
                return []

    def __iter__(self):
        def _traverse(prefix_sequence, trie_dict):
            if trie_dict:
                for next_token in trie_dict:
                    yield from _traverse(
                        prefix_sequence + [next_token], trie_dict[next_token]
                    )
            else:
                yield prefix_sequence

        return _traverse([], self.trie_dict)

    def __len__(self):
        return self.len

    def __getitem__(self, value):
        return self.get(value)


In [21]:
allowed_gen_sequences = []
with open('../modelling/datasets/taxonomy/wish_v1.2.1_newtax_leafpaths.txt', 'r') as f:
    for l in f:
        l = l.replace('\n', '').strip()
        if len(l) > 0:
            allowed_gen_sequences.append(l)

trie = Trie([
    [tokenizer.pad_token_id] + tokenizer.encode(i) + [tokenizer.eos_token_id] for i in allowed_gen_sequences
])

def constraint(batch_id, sent):
    return trie.get(sent.tolist())


In [22]:
batch = inputs
batch2 = inputs2
batch3 = inputs3

## constrained beam-search

In [23]:
model.generate3 = torchdynamo.optimize("inductor")(model.generate)

In [24]:
infres = model.generate3(
    input_ids = batch["input_ids"], 
    attention_mask = batch["attention_mask"],
    num_beams = 3, 
    num_return_sequences = 3, 
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint, 
    output_scores=True, return_dict_in_generate=True
)
prediction = infres.sequences
probs = infres.sequences_scores.exp()
print(tokenizer.batch_decode(prediction))
print(probs)

[2023-01-31 17:35:56,878] torch._dynamo.symbolic_convert: [WARNING] /tmp/ipykernel_19447/4251114661.py <function constraint at 0x7f27a5556440> [UserDefinedObjectVariable(PrefixConstrainedLogitsProcessor), ConstantVariable(int), TensorVariable()] {} too many positional arguments


['<pad> beauty & health > sexual wellness > sex toys > dildos</s>', '<pad> beauty & health > sexual wellness > sm products > masturbation machine</s>', '<pad> beauty & health > sexual wellness > safer sex > condoms</s>']
tensor([9.3622e-01, 1.1616e-03, 8.2010e-04], device='cuda:0')


In [26]:
%%timeit
infres = model.generate3(
    input_ids = batch["input_ids"], 
    attention_mask = batch["attention_mask"],
    num_beams = 3, 
    num_return_sequences = 3, 
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint, 
    output_scores=True, return_dict_in_generate=True
)
prediction = infres.sequences
probs = infres.sequences_scores.exp()
print(tokenizer.batch_decode(prediction))
print(probs)

['<pad> beauty & health > sexual wellness > sex toys > dildos</s>', '<pad> beauty & health > sexual wellness > sm products > masturbation machine</s>', '<pad> beauty & health > sexual wellness > safer sex > condoms</s>']
tensor([9.3622e-01, 1.1616e-03, 8.2010e-04], device='cuda:0')
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>', '<pad> beauty & health > sexual wellness > sm products > masturbation machine</s>', '<pad> beauty & health > sexual wellness > safer sex > condoms</s>']
tensor([9.3622e-01, 1.1616e-03, 8.2010e-04], device='cuda:0')
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>', '<pad> beauty & health > sexual wellness > sm products > masturbation machine</s>', '<pad> beauty & health > sexual wellness > safer sex > condoms</s>']
tensor([9.3622e-01, 1.1616e-03, 8.2010e-04], device='cuda:0')
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>', '<pad> beauty & health > sexual wellness > sm products > masturbation machine</s>',

In [27]:
%%timeit
infres = model.generate3(
    input_ids = batch2["input_ids"], 
    attention_mask = batch2["attention_mask"],
    num_beams = 3, 
    num_return_sequences = 3, 
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint, 
    output_scores=True, return_dict_in_generate=True
)
prediction = infres.sequences
probs = infres.sequences_scores.exp()
print(tokenizer.batch_decode(prediction))
print(probs)

["<pad> luggage & bags > women's bags > shoulder bags</s><pad>", "<pad> luggage & bags > women's bags > top-handle bags</s>", "<pad> women's clothing > tops > tees</s><pad><pad><pad><pad><pad>"]
tensor([0.1285, 0.0830, 0.0501], device='cuda:0')
["<pad> luggage & bags > women's bags > shoulder bags</s><pad>", "<pad> luggage & bags > women's bags > top-handle bags</s>", "<pad> women's clothing > tops > tees</s><pad><pad><pad><pad><pad>"]
tensor([0.1285, 0.0830, 0.0501], device='cuda:0')
["<pad> luggage & bags > women's bags > shoulder bags</s><pad>", "<pad> luggage & bags > women's bags > top-handle bags</s>", "<pad> women's clothing > tops > tees</s><pad><pad><pad><pad><pad>"]
tensor([0.1285, 0.0830, 0.0501], device='cuda:0')
["<pad> luggage & bags > women's bags > shoulder bags</s><pad>", "<pad> luggage & bags > women's bags > top-handle bags</s>", "<pad> women's clothing > tops > tees</s><pad><pad><pad><pad><pad>"]
tensor([0.1285, 0.0830, 0.0501], device='cuda:0')
["<pad> luggage & ba

In [28]:
%%timeit
infres = model.generate3(
    input_ids = batch3["input_ids"], 
    attention_mask = batch3["attention_mask"],
    num_beams = 3, 
    num_return_sequences = 3, 
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint, 
    output_scores=True, return_dict_in_generate=True
)
prediction = infres.sequences
probs = infres.sequences_scores.exp()
print(tokenizer.batch_decode(prediction))
print(probs)

['<pad> mother & kids > feeding > solid feeding > cups</s><pad><pad>', '<pad> mother & kids > feeding > baby food storage</s><pad><pad><pad><pad><pad>', '<pad> mother & kids > feeding > bottle feeding > formula milk storage</s>']
tensor([0.1415, 0.1347, 0.0729], device='cuda:0')
['<pad> mother & kids > feeding > solid feeding > cups</s><pad><pad>', '<pad> mother & kids > feeding > baby food storage</s><pad><pad><pad><pad><pad>', '<pad> mother & kids > feeding > bottle feeding > formula milk storage</s>']
tensor([0.1415, 0.1347, 0.0729], device='cuda:0')
['<pad> mother & kids > feeding > solid feeding > cups</s><pad><pad>', '<pad> mother & kids > feeding > baby food storage</s><pad><pad><pad><pad><pad>', '<pad> mother & kids > feeding > bottle feeding > formula milk storage</s>']
tensor([0.1415, 0.1347, 0.0729], device='cuda:0')
['<pad> mother & kids > feeding > solid feeding > cups</s><pad><pad>', '<pad> mother & kids > feeding > baby food storage</s><pad><pad><pad><pad><pad>', '<pad> 

## constrained greedy decoding

In [30]:
model.generate4 = torchdynamo.optimize("inductor")(model.generate)

In [31]:
print(tokenizer.batch_decode(model.generate4(
    input_ids = batch["input_ids"], 
    attention_mask = batch["attention_mask"],
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint
)))

['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']


In [35]:
%%timeit
print(tokenizer.batch_decode(model.generate4(
    input_ids = batch["input_ids"], 
    attention_mask = batch["attention_mask"],
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint
)))

['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex toys > dildos</s>']
['<pad> beauty & health > sexual wellness > sex 

In [33]:
%%timeit
print(tokenizer.batch_decode(model.generate4(
    input_ids = batch2["input_ids"], 
    attention_mask = batch2["attention_mask"],
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint
)))

["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > shoulder bags</s>"]
["<pad> luggage & bags > women's bags > 

In [37]:
%%timeit
print(tokenizer.batch_decode(model.generate4(
    input_ids = batch3["input_ids"], 
    attention_mask = batch3["attention_mask"],
    do_sample = False, 
    length_penalty = 0, 
    max_new_tokens = 50 - 1, # HACK: T5 adds pad token in the beginning
    prefix_allowed_tokens_fn=constraint
)))

['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feeding > bottle feeding > baby bottle accessories</s>']
['<pad> mother & kids > feed